In [21]:
import cv2
import numpy as np
import dlib
import pandas as pd
import os, glob
from imutils import face_utils
import argparse
import imutils
import pickle

In [2]:
def extract_features (path):
    
    # Load the detector
    detector = dlib.get_frontal_face_detector()
    
    # Load the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    emotion_codes = {
                    'Angry': 0,
                    'Disgust' : 1, 
                    'Fear' : 2, 
                    'Happy' : 3, 
                    'Sad' : 4, 
                    'Surprise' : 5, 
                    'Neutral': 6
                    }
    emotions_features = pd.DataFrame()
    
    for filepath in glob.glob(path):
        
        #Split a path to get emotion name
        emotion = filepath.split(os.sep)[1]
    
        # Match dictionary key format from folder name
        emotion_code = emotion_codes[emotion.capitalize()]
        
        
    # for filepath in glob.glob("Training_803556.jpg"):
        
        # Read the image
        img = cv2.imread(filepath)
    
        # Convert image into grayscale
        gray = cv2.cvtColor(src=img, code=cv2.COLOR_BGR2GRAY)

        # Use detector to find landmarks
        faces = detector(gray)
        
        for face in faces:
            x1 = face.left() # left point
            y1 = face.top() # top point
            x2 = face.right() # right point
            y2 = face.bottom() # bottom point


            landmarks = predictor(image=gray, box=face)
            
            emptyarray = []
            
            for n in range(0, 68):
                x = landmarks.part(n).x
                y = landmarks.part(n).y

                # Created a coordinate numpy array
                coordinates = np.array([x, y])
            
                # Append coordinate values (x, y) to emptyarray
                emptyarray.append(coordinates[0])
                emptyarray.append(coordinates[1])

                # Create dataframe
                df = pd.DataFrame({'emotion_code': emotion_code, 'emotion':emotion, 'features':[emptyarray]})
        
            # Append all df from all files/faces
            emotions_features = emotions_features.append(df)
        
    emotions_features['features']= emotions_features['features'].astype(str).str[1:-1]
    emotions_features2 = emotions_features['features'].str.split(",", n = 135, expand = True) 
    emotions_features2.rename(columns= lambda s:f"data{s}", inplace=True )

    emotions_features2 = emotions_features2.apply(pd.to_numeric)
    emotions_features2.fillna(0,inplace=True)
    emotions_features2 = emotions_features2.astype({"data0": int})

    emotions_features_final = pd.concat([emotions_features[['emotion_code','emotion']],emotions_features2], axis=1, join="inner")
    emotions_features_final.reset_index(drop=True, inplace=True)

    return emotions_features_final 

In [3]:
# Extract labeled dataset features to train model
path = "emotions_dataset\\**\\*.jpg"
df = extract_features(path)
df.head()

,emotion_code,emotion,data0,data1,data2,data3,data4,data5,data6,data7,...,data126,data127,data128,data129,data130,data131,data132,data133,data134,data135
0,0,angry,68,77,64,96,62,115,62,133,...,144,156,153,157,143,155,138,154,131,153
1,0,angry,12,89,23,109,34,127,49,143,...,138,94,146,93,140,101,136,104,131,107
2,0,angry,18,76,22,98,29,119,37,139,...,113,126,130,133,114,143,106,144,97,144
3,0,angry,30,77,33,98,39,119,46,138,...,105,153,125,156,106,165,97,168,88,168
4,0,angry,22,111,27,129,32,147,37,164,...,112,157,131,157,114,159,106,161,97,162


In [4]:
def file_features(path):
    
    # Load the detector
    detector = dlib.get_frontal_face_detector()
    
    # Load the predictor
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    emotions_features = pd.DataFrame()
    
    for filepath in glob.glob(path):
        
        # Read the image
        img = cv2.imread(filepath)
    
        # Convert image into grayscale
        gray = cv2.cvtColor(src=img, code=cv2.COLOR_BGR2GRAY)

        # Use detector to find landmarks
        faces = detector(gray)
        
        for face in faces:
            x1 = face.left() # left point
            y1 = face.top() # top point
            x2 = face.right() # right point
            y2 = face.bottom() # bottom point


            landmarks = predictor(image=gray, box=face)
            
            emptyarray = []
            
            for n in range(0, 68):
                x = landmarks.part(n).x
                y = landmarks.part(n).y
    #             print(x)
                # Created a coordinate numpy array
                coordinates = np.array([x, y])
            
                # Append coordinate values (x, y) to emptyarray
                emptyarray.append(coordinates[0])
                emptyarray.append(coordinates[1])

                # Create dataframe
                df = pd.DataFrame({'features':[emptyarray]})
        
            # Append all df from all files/faces
            emotions_features = emotions_features.append(df)
        
    emotions_features['features']= emotions_features['features'].astype(str).str[1:-1]
    emotions_features2 = emotions_features['features'].str.split(",", n = 135, expand = True) 
    emotions_features2.rename(columns= lambda s:f"data{s}", inplace=True )

    emotions_features2 = emotions_features2.apply(pd.to_numeric)
    emotions_features2.fillna(0,inplace=True)
    emotions_features2 = emotions_features2.astype({"data0": int})

    
    emotions_features2.reset_index(drop=True, inplace=True)

    return emotions_features2

In [6]:
# Testing the file_features built in function for a single file
# path = "face.png"
# df2 = file_features(path)
# df2

# Data Pre-Processing

In [13]:
def process_data(df):
    import pickle
    
    X = df.drop(['emotion_code', 'emotion'], axis = 1)
    y = df['emotion']
    
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder, MinMaxScaler
    from tensorflow.keras.utils import to_categorical

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    
    X_scaler = MinMaxScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    
    # Step 1: Label-encode data set
    label_encoder = LabelEncoder()
    label_encoder.fit(y_train)
    encoded_y_train = label_encoder.transform(y_train)
    encoded_y_test = label_encoder.transform(y_test)
    
    # Step 2: Convert encoded labels to one-hot-encoding
    y_train_categorical = to_categorical(encoded_y_train)
    y_test_categorical = to_categorical(encoded_y_test)
    
    # Saving Label Encoder
    with open('label_encoder.pk', 'wb') as fin:
        pickle.dump(label_encoder, fin)
    
    return X_train_scaled, y_train_categorical, X_test_scaled, y_test_categorical, y_test, label_encoder  

## Create a Deep Learning Model

In [14]:
def create_dl_model(X_train_scaled, y_train_categorical, X_test_scaled, y_test, label_encoder):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    from sklearn.preprocessing import LabelEncoder
    
    # Create model and add layers
    model = Sequential()
    model.add(Dense(units=200, activation='relu', input_dim=136))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=200, activation='relu'))
    model.add(Dense(units=7, activation='softmax'))
    
    # Compile and fit the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Fit training data to model
    model.fit(
        X_train_scaled,
        y_train_categorical,
        epochs=600,
        shuffle=True,
        verbose=2
        )
    
    # Make Predictions
    encoded_predictions = model.predict_classes(X_test_scaled[:100])
    prediction_labels = label_encoder.inverse_transform(encoded_predictions)
    
    result_df = pd.DataFrame()
    result_df['Predicted'] = prediction_labels
    result_df['Actual'] = list(y_test[:100])
    
    # Save model
    model.save("image_emotions_model.h5")
    
    return model, result_df

In [15]:
def make_prediction(model, path):
    
    # Extract image file features
    df = file_features(path)
    
    # Make prediction using imported model
    prediction = model.predict_classes(df)
    prediction_label = label_encoder.inverse_transform(prediction)
    
#     # Emotion labels
#     emotion_codes = {
#                 0: 'Angry',
#                 1:'Disgust' ,
#                 2:'Fear',
#                 3:'Happy',
#                 4:'Sad',
#                 5:'Surprise',
#                 6:'Neutral'
#                }
#     result = emotion_codes[prediction[0]]
    
#     return result    
    return prediction_label[0]

In [16]:
# Process my data extracted from image
X_train, y_train, X_test, y_test_categorical, y_test, label_encoder = process_data(df)

In [17]:
X_train.shape

(15852, 136)

In [18]:
# from tensorflow.keras.models import load_model
# model =load_model("image_emotions_model.h5")

# Create Model with processed data
model, results_df = create_dl_model(X_train, y_train, X_test, y_test, label_encoder)

Train on 15852 samples
Epoch 1/600
15852/15852 - 2s - loss: 1.7599 - accuracy: 0.3000
Epoch 2/600
15852/15852 - 2s - loss: 1.5753 - accuracy: 0.3971
Epoch 3/600
15852/15852 - 2s - loss: 1.5303 - accuracy: 0.4194
Epoch 4/600
15852/15852 - 2s - loss: 1.4742 - accuracy: 0.4428
Epoch 5/600
15852/15852 - 2s - loss: 1.4658 - accuracy: 0.4432
Epoch 6/600
15852/15852 - 2s - loss: 1.4327 - accuracy: 0.4537
Epoch 7/600
15852/15852 - 2s - loss: 1.4220 - accuracy: 0.4618
Epoch 8/600
15852/15852 - 2s - loss: 1.4151 - accuracy: 0.4591
Epoch 9/600
15852/15852 - 2s - loss: 1.4076 - accuracy: 0.4640
Epoch 10/600
15852/15852 - 2s - loss: 1.4005 - accuracy: 0.4683
Epoch 11/600
15852/15852 - 2s - loss: 1.4064 - accuracy: 0.4608
Epoch 12/600
15852/15852 - 2s - loss: 1.3856 - accuracy: 0.4724
Epoch 13/600
15852/15852 - 2s - loss: 1.3939 - accuracy: 0.4669
Epoch 14/600
15852/15852 - 2s - loss: 1.3872 - accuracy: 0.4709
Epoch 15/600
15852/15852 - 2s - loss: 1.3818 - accuracy: 0.4753
Epoch 16/600
15852/15852 -

Epoch 129/600
15852/15852 - 2s - loss: 1.0789 - accuracy: 0.5880
Epoch 130/600
15852/15852 - 2s - loss: 1.0519 - accuracy: 0.5999
Epoch 131/600
15852/15852 - 2s - loss: 1.0586 - accuracy: 0.5934
Epoch 132/600
15852/15852 - 2s - loss: 1.0530 - accuracy: 0.5953
Epoch 133/600
15852/15852 - 2s - loss: 1.0346 - accuracy: 0.6026
Epoch 134/600
15852/15852 - 2s - loss: 1.0593 - accuracy: 0.5975
Epoch 135/600
15852/15852 - 2s - loss: 1.0547 - accuracy: 0.6004
Epoch 136/600
15852/15852 - 2s - loss: 1.0419 - accuracy: 0.6010
Epoch 137/600
15852/15852 - 2s - loss: 1.0486 - accuracy: 0.5986
Epoch 138/600
15852/15852 - 2s - loss: 1.0487 - accuracy: 0.6005
Epoch 139/600
15852/15852 - 2s - loss: 1.0288 - accuracy: 0.6085
Epoch 140/600
15852/15852 - 2s - loss: 1.0425 - accuracy: 0.5995
Epoch 141/600
15852/15852 - 2s - loss: 1.0363 - accuracy: 0.6064
Epoch 142/600
15852/15852 - 2s - loss: 1.0401 - accuracy: 0.6036
Epoch 143/600
15852/15852 - 2s - loss: 1.0360 - accuracy: 0.6057
Epoch 144/600
15852/15852

15852/15852 - 2s - loss: 0.6373 - accuracy: 0.7614
Epoch 256/600
15852/15852 - 2s - loss: 0.6616 - accuracy: 0.7519
Epoch 257/600
15852/15852 - 2s - loss: 0.6249 - accuracy: 0.7658
Epoch 258/600
15852/15852 - 2s - loss: 0.6240 - accuracy: 0.7622
Epoch 259/600
15852/15852 - 2s - loss: 0.6867 - accuracy: 0.7463
Epoch 260/600
15852/15852 - 2s - loss: 0.6238 - accuracy: 0.7646
Epoch 261/600
15852/15852 - 2s - loss: 0.6695 - accuracy: 0.7514
Epoch 262/600
15852/15852 - 2s - loss: 0.6626 - accuracy: 0.7535
Epoch 263/600
15852/15852 - 2s - loss: 0.5870 - accuracy: 0.7757
Epoch 264/600
15852/15852 - 2s - loss: 0.7101 - accuracy: 0.7373
Epoch 265/600
15852/15852 - 2s - loss: 0.6335 - accuracy: 0.7604
Epoch 266/600
15852/15852 - 2s - loss: 0.5818 - accuracy: 0.7822
Epoch 267/600
15852/15852 - 2s - loss: 0.6525 - accuracy: 0.7561
Epoch 268/600
15852/15852 - 2s - loss: 0.6365 - accuracy: 0.7612
Epoch 269/600
15852/15852 - 2s - loss: 0.5705 - accuracy: 0.7843
Epoch 270/600
15852/15852 - 2s - loss: 

Epoch 382/600
15852/15852 - 2s - loss: 0.4116 - accuracy: 0.8505
Epoch 383/600
15852/15852 - 2s - loss: 0.3897 - accuracy: 0.8577
Epoch 384/600
15852/15852 - 2s - loss: 0.4131 - accuracy: 0.8488
Epoch 385/600
15852/15852 - 2s - loss: 0.3916 - accuracy: 0.8583
Epoch 386/600
15852/15852 - 2s - loss: 0.4043 - accuracy: 0.8518
Epoch 387/600
15852/15852 - 2s - loss: 0.3869 - accuracy: 0.8625
Epoch 388/600
15852/15852 - 2s - loss: 0.4065 - accuracy: 0.8525
Epoch 389/600
15852/15852 - 2s - loss: 0.4385 - accuracy: 0.8446
Epoch 390/600
15852/15852 - 2s - loss: 0.4195 - accuracy: 0.8482
Epoch 391/600
15852/15852 - 2s - loss: 0.3942 - accuracy: 0.8583
Epoch 392/600
15852/15852 - 2s - loss: 0.4231 - accuracy: 0.8434
Epoch 393/600
15852/15852 - 2s - loss: 0.4013 - accuracy: 0.8564
Epoch 394/600
15852/15852 - 2s - loss: 0.4501 - accuracy: 0.8373
Epoch 395/600
15852/15852 - 2s - loss: 0.4370 - accuracy: 0.8420
Epoch 396/600
15852/15852 - 2s - loss: 0.3626 - accuracy: 0.8702
Epoch 397/600
15852/15852

15852/15852 - 2s - loss: 0.3975 - accuracy: 0.8678
Epoch 509/600
15852/15852 - 2s - loss: 0.2892 - accuracy: 0.9008
Epoch 510/600
15852/15852 - 2s - loss: 0.3351 - accuracy: 0.8881
Epoch 511/600
15852/15852 - 2s - loss: 0.3267 - accuracy: 0.8892
Epoch 512/600
15852/15852 - 2s - loss: 0.2660 - accuracy: 0.9067
Epoch 513/600
15852/15852 - 2s - loss: 0.3042 - accuracy: 0.8924
Epoch 514/600
15852/15852 - 3s - loss: 0.2843 - accuracy: 0.8994
Epoch 515/600
15852/15852 - 3s - loss: 0.2801 - accuracy: 0.9013
Epoch 516/600
15852/15852 - 3s - loss: 0.3174 - accuracy: 0.8894
Epoch 517/600
15852/15852 - 2s - loss: 0.2938 - accuracy: 0.8991
Epoch 518/600
15852/15852 - 2s - loss: 0.2315 - accuracy: 0.9194
Epoch 519/600
15852/15852 - 2s - loss: 0.3402 - accuracy: 0.8827
Epoch 520/600
15852/15852 - 2s - loss: 0.3437 - accuracy: 0.8827
Epoch 521/600
15852/15852 - 2s - loss: 0.2630 - accuracy: 0.9081
Epoch 522/600
15852/15852 - 2s - loss: 0.3953 - accuracy: 0.8678
Epoch 523/600
15852/15852 - 2s - loss: 

In [23]:
# Make predictions
img_path = 'songsad_post.jpg'
test_img = make_prediction(model, img_path)
test_img.capitalize()

'Sad'

In [26]:
# # Saving Label Encoder
# with open('label_encoder.pk', 'wb') as fin:
#     pickle.dump(label_encoder, fin)